In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [18]:
cities_df = pd.read_csv("Project_2/cities.csv")
cities_df.head()

,City,Wiki_String
0,Birmingham,"Birmingham,_Alabama"
1,Huntsville,"Huntsville,_Alabama"
2,Montgomery,"Montgomery,_Alabama"
3,Mobile,"Mobile,_Alabama"
4,Tuscaloosa,"Tuscaloosa,_Alabama"


In [19]:
w_string=np.asarray(cities_df['Wiki_String'])
cities = np.asarray(cities_df['City'])

In [20]:
count = 0

In [21]:
wdf = pd.DataFrame()

In [22]:
for city in w_string:
    
    City_Name = cities[count]
    
    if count == 1:
        print("Beginning Data Retrieval")
        print("---------------------------------")
    
    try:
        print(f"Processing record {count}: City Name={City_Name}")
        url = 'https://en.wikipedia.org/wiki/' + city
        html = requests.get(url).content
        soup = BeautifulSoup(html)
        table = soup.select_one('table:contains("Climate data")')
        df_table = pd.read_html(str(table))
        df = df_table[0]
        df.columns = ["Measure", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "Year"]
    
        High_temp = df.set_index("Measure").filter(like='Average high', axis=0)
        Low_temp = df.set_index("Measure").filter(like='Average low', axis=0)
        Precip_temp = df.set_index("Measure").filter(like='Average precipitation inches', axis=0)
        
        if Precip_temp.size == 0:
            Precip_temp = df.set_index("Measure").filter(like="Average rainfall inches", axis=0)
    
        df1 = High_temp.append(Low_temp)
        df2 = df1.append(Precip_temp)
    
        City_Name = cities[count]
        df2['City'] = City_Name
    
        wdf = wdf.append(df2)
        
    except (KeyError, IndexError, ValueError):
        print(f"Processing record {count}: Missing city...skipping.")
    
    if count == len(cities):
        print("---------------------------------")
        print("Data Retrieval Complete")
        print("---------------------------------")
    
    count = count+1
    

Processing record 0: City Name=Birmingham 
Beginning Data Retrieval
---------------------------------
Processing record 1: City Name=Huntsville 
Processing record 2: City Name=Montgomery 
Processing record 3: City Name=Mobile 
Processing record 4: City Name=Tuscaloosa 
Processing record 5: City Name=Anchorage 
Processing record 6: City Name=Phoenix 
Processing record 7: City Name=Tucson 
Processing record 8: City Name=Mesa 
Processing record 9: City Name=Scottsdale 
Processing record 10: City Name=Flagstaff
Processing record 11: City Name=Sedona
Processing record 12: City Name=Little Rock 
Processing record 13: City Name=Los Angeles 
Processing record 14: City Name=San Diego 
Processing record 15: City Name=San Jose 
Processing record 16: City Name=San Francisco 
Processing record 17: City Name=Fresno 
Processing record 18: City Name=Sacramento 
Processing record 19: City Name=Long Beach 
Processing record 20: City Name=Oakland 
Processing record 21: City Name=South Lake Tahoe
Processi

In [25]:
output_data_file = "Project_2/city_weather.csv"
wdf.to_csv(output_data_file, index=False, header=True)